<a href="https://colab.research.google.com/github/SpandanaKalakonda/Building_ML_Pipelines/blob/main/counting_words_using_apache_beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install apache-beam

In [20]:
import re
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [21]:
input_file = "/content/kinglear.txt"
output_file = "/content/output.txt"

In [22]:
pipeline_options = PipelineOptions()

In [23]:
with beam.Pipeline(options = pipeline_options) as p:
  lines = (
  p
  | beam.io.ReadFromText(input_file , coder=beam.coders.BytesCoder())
  )
  decoded_lines = lines | beam.Map(lambda line: line.decode('utf-8'))
  counts = (
    decoded_lines
    | 'Split' >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x))
    | 'PairWithOne' >> beam.Map(lambda x: (x,1))
    | 'GroupAndSum' >> beam.CombinePerKey(sum)
    )

  def format_result(word_count):
    """Convert tuples (token, count) into a string"""
    (word, count) = word_count
    return "{}: {}".format(word, count)

  output = counts | 'Format' >> beam.Map(format_result)

  output | WriteToText(output_file)
